In [3]:
import os
import sys
import h5py
import json
import numpy as np
import torch as pt
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

from src.dataset import StructuresDataset, collate_batch_features, select_by_sid, select_by_interface_types
from src.data_encoding import encode_structure, encode_features, extract_topology, categ_to_resnames, resname_to_categ
from src.structure import data_to_structure, encode_bfactor, concatenate_chains, split_by_chain
from src.structure_io import save_pdb, read_pdb
from src.scoring import bc_scoring, bc_score_names

# data parameters
data_path = "/your/PDB/file/Path"

# model parameters
# R3
#save_path = "model/save/i_v3_0_2021-05-27_14-27"  # 89
#save_path = "model/save/i_v3_1_2021-05-28_12-40"  # 90
# R4
#save_path = "model/save/i_v4_0_2021-09-07_11-20"  # 89
save_path = "model/save/i_v4_1_2021-09-07_11-21"  # 91
# save_path = "/mnt/disk1/guoxiaokun/isoform/PeSTO/PeSTo-main/examples/isoform/save"
# select saved model
model_filepath = os.path.join(save_path, 'model_ckpt.pt')
#model_filepath = os.path.join(save_path, 'model.pt')

# add module to path
if save_path not in sys.path:
    sys.path.insert(0, save_path)

In [6]:
# load functions
from config import config_model, config_data
from data_handler import Dataset
from model import Model

device = pt.device("cpu")
model = Model(config_model)
model.load_state_dict(pt.load(model_filepath, map_location=pt.device("cpu")))
model = model.eval().to(device)
# find pdb files and ignore already predicted oins
pdb_filepaths = glob(os.path.join(data_path, "*.pdb"), recursive=True)
pdb_filepaths = [fp for fp in pdb_filepaths if "_i" not in fp]
# create dataset loader with preprocessing
dataset = StructuresDataset(pdb_filepaths, with_preprocessing=True)


In [ ]:
import numpy as np
from Bio.PDB import *
from Bio.PDB.DSSP import DSSP
from itertools import chain
import pickle

protein_dict = {}
ss_mapping = {'H': 0, 'G':0,'I':0,'E':1,'T':2,'B':2,'S':2,'-':3}
with pt.no_grad():
    for subunits, filepath in tqdm(dataset):
        structure = concatenate_chains(subunits)
        X, M = encode_structure(structure)
        #q = pt.cat(encode_features(structure), dim=1)
        q = encode_features(structure)[0]
        # extract topology
        ids_topk, _, _, _, _ = extract_topology(X, 64)
        # pack data and setup sink (IMPORTANT)
        X, ids_topk, q, M = collate_batch_features([[X, ids_topk, q, M]])
        z = model(X.to(device), ids_topk.to(device), q.to(device), M.float().to(device))
        element = filepath[:-4].split('/')[-1]
        proval = pt.sigmoid(z[:,:]).tolist()
        # PDBParser
        parser = PDBParser(QUIET=True)
        structure_PDB = parser.get_structure("protein", filepath)
        # DSSP
        model_PDB = structure_PDB[0]
        dssp = DSSP(model_PDB, filepath)
        codes = [] 
        for residue in dssp:
            res_id = residue[1]  
            ss = residue[2]
            code = ss_mapping[ss]  
            # print(f"Residue {res_id}: Predicted Secondary Structure : {ss}  code:{code}  count:{count}")
            codes.append(code)
        merge = np.hstack((proval, np.array([codes]).T))
        protein_dict[element] = merge

with open('/mnt/disk1/guoxiaokun/isoform/DeepISO/BRCA/BC.geometirc.allPeSTO.sencond.simple.feature.pkl', 'wb') as file:
    pickle.dump(protein_dict, file)